In [1]:
from models.DAX import DAX_quantile_regression, DAX_baseline
from models.energy import energy_quantile_regression
from functions import get_energy,get_DAX,merge_submissions,check_submission
from functions.prepare_data import split_time
from functions.evaluation import evaluate_horizon
from tqdm import tqdm
from functions import evaluate_dax

In [2]:
from functions import naive_ensemble

In [3]:
daxdata = get_DAX.get()

In [4]:
base = DAX_baseline.DAX_baseline(daxdata)

In [5]:
quantile = DAX_quantile_regression.DAX_quantile_regression(daxdata)

In [7]:
naive_ensemble.ensemble(base,quantile,0.5,0.5)

,q0.025,q0.25,q0.5,q0.75,q0.975,forecast_date,target,horizon
0,-2.815561,-0.530334,0.048978,0.622808,2.244557,2023-11-27,DAX,1 day
1,-3.104939,-0.545643,0.230696,0.941829,3.009623,2023-11-27,DAX,2 day
2,-3.556624,-0.434429,0.458940,1.242344,3.669568,2023-11-27,DAX,5 day
3,-3.805432,-0.212801,0.780258,1.676015,4.366634,2023-11-27,DAX,6 day
4,-4.010356,-0.270559,0.808544,1.694942,4.838923,2023-11-27,DAX,7 day
